# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 데이터 결측치 처리하기
- 파생변수 생성하기

### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [43]:
# basic
import os, random
import pandas as pd
import numpy as np

# graph
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# imputator
from sklearn.impute import KNNImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 폰트
plt.rcParams['font.family'] = 'NanumSquare'

# 마이너스 출력
matplotlib.rcParams['axes.unicode_minus'] = False

In [44]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [45]:
# load data
train = pd.read_csv('../data/fog_train.csv')
test = pd.read_csv('../data/fog_test.csv')

In [46]:
# 쓸데없는 열 제거
train.drop(['Unnamed: 0'], axis = 1, inplace = True)
test.drop(['Unnamed: 0'], axis = 1, inplace = True)


### 3. 데이터 결측치 처리하기

In [47]:
# train 없는 날짜 채우기
train = train.append([{'fog_train.year':'J',
                'fog_train.month':11, 
                'fog_train.day':3,
                'fog_train.time':18, 
                'fog_train.minute':20,
                'fog_train.stn_id':'EC'}], ignore_index=True, sort=True)

In [48]:
# 결측치 nan으로 바꾸기
train2 = train.replace(-99, np.nan).replace(-99.9, np.nan).replace(-999.0, np.nan)
test2 = test.replace(-99, np.nan).replace(-99.9, np.nan).replace(-999.0, np.nan)

In [49]:
# 변수 이름 앞에있는 이상한거 제거하기
train2.columns = train2.columns.str.replace('fog_train.', '')
test2.columns = test2.columns.str.replace('fog_test.', '')

#### 3-1) 지역의 유형별로 나누기

In [50]:
# A:내륙, B:내륙산간, C:동해, D:서해, E:남해
train2['ground'] = train2['stn_id'].str.slice(0, 1)
test2['ground'] = test2['stn_id'].str.slice(0, 1)

#### 3-2) 연도를 적합하기 좋은 연도로 바꿔주기
- 그래프로 그려보기 편하게 I -> 2020 ~ K -> 2022
- 빠르게 바꾸도록 하자

In [51]:
# 시간
train2['hour'] = train2['time']

# I, J, K -> 2020 ~ 2022
cri = [
    train2['year'] == "I",
    train2['year'] == "J"
]
con = [
    2020,
    2021
]
train2['yeartmp'] = np.select(cri, con, default = 2022)

# year 값 바꾸기
train3 = train2.copy()
train3['year'] = train3['yeartmp']
train3.drop(['yeartmp'], axis = 1, inplace = True)

# 연, 월, 일, 시간, 분을 하나의 datetime 객체로 변환하고 문자열 형식으로 변환
train3['DateTime'] = pd.to_datetime(train3[['year', 'month', 'day', 'hour', 'minute']]).dt.strftime('%Y-%m-%d %H:%M')

#### 3-3) 이상치 대치

- 클래스가 뾰족하게 4 -> 1 -> 4 와 같이 분포하고 있는 것들이 몇개 있다.
- 이를 보완하기 위해 다음과 같은 과정을 진행한다.
- 단, 1000 이하를 중심으로 맞춰야하는 대회이기 때문에 class를 기준으로 적용하였다.

In [52]:
# # 지역별로 따로따로 적용하자.
# for c in train3['stn_id'].unique():

#     # 지역별로 잘라서 이상치 -> 결측치 만들기
#     train4 = train3[train3['stn_id'] == c]

#     # lag 만들어서 지난번  class 확인하기
#     train4['shift_left_class'] = train4['class'].shift(1)

#     # 이전과의 차이 구하기
#     train4['diff_left_class'] = train4['class'] - train4['shift_left_class']

#     # lag 만들어서 이다음 class 확인하기
#     train4['shift_right_class'] = train4['class'].shift(-1)

#     # 이다음과의 차이 구하기
#     train4['diff_right_class'] = train4['class'] - train4['shift_right_class']

#     # 인덱스 뽑기
#     idx = train4[(abs(train4['diff_left_class']) >= 2) & (abs(train4['diff_right_class']) >= 2)].index

#     # 뾰족한 데이터 결측치로 처리하기
#     train3['class'].iloc[idx] = np.nan
#     train3['vis1'].iloc[idx] = np.nan

#### 3-4) KNN Imputation

- 서로간의 근접 이웃을 통해 값을 유사하게 채워주는 보간법이다.
- 내륙, 해안, 산간지방의 5가지 유형을 통해 분해하여 따로 처리해주는 방식을 사용한다.

- 다음과 같은 방법으로 결측치를 채우도록 하자

> 1. vis1을 제외한 train으로 knnimputator을 적용한 다음 test에 유형에 맞춰 값을 대치한다.
>    - 주의할점 : 시간에 관련된 변수는 제외한다. 값을 반영함으로 오히려 방해될 수 있다.
> 2. 대치된 값을 활용하여 vis1을 만들고 class를 채운다.

In [67]:
# imputate with knnimputer
for c in ['A']: # , 'E', 'B', 'C', 'D'
    
    print(f'The first processing: {c} ground transforming ...')
    # 1. train & test knn imputer
    knnimputer = KNNImputer()

    # train - fit_transform
    print("train transforming ....")
    train3.loc[train3['ground'] == c, ['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm']] = knnimputer.fit_transform(train3[train3['ground'] == c][['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm']])

    # test - transform
    print("test transforming ....")
    test2.loc[test2['ground'] == c, ['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm']] = knnimputer.transform(test2[test2['ground'] == c][['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm']])
    
    # 2. vis1 imputater
    visimputer = KNNImputer()

    print(f'The second processing: {c} vis1 transforming ...')
    # only train
    train3.loc[train3['ground'] == c, ['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'vis1']] = knnimputer.fit_transform(train3[train3['ground'] == c][['re', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'hm', 'vis1']])
    print(f'The end of {c} ground imputate. check this train data')

The first processing: A ground transforming ...
train transforming ....
test transforming ....
The second processing: A vis1 transforming ...
The end of A ground imputate. check this train data


In [68]:
train3.isna().sum()

class       22559
day             0
hm              0
minute          0
month           0
re              0
stn_id          0
sun10           0
ta              0
time            0
ts              0
vis1            0
ws10_deg        0
ws10_ms         0
year            0
ground          0
hour            0
DateTime        0
dtype: int64

#### 3-5) class 계산하기

- vis1을 활용하면 class가 계산된다.
- 실제로 기준의 범위를 나눌때 vis1 변수를 활용하기 때문에 이번에도 이와 같이 반영해준다.

In [88]:
# vis1을 활용하여 class 계산해주기
# 기준
cri = [
    (train3['class'].isna()) & (0 < train3['vis1']) & (train3['vis1'] < 200),
    (train3['class'].isna()) & (200 <= train3['vis1']) & (train3['vis1'] < 500),
    (train3['class'].isna()) & (500 <= train3['vis1']) & (train3['vis1'] < 1000),
    (train3['class'].isna()) & (1000 <= train3['vis1'])
]

# 반영값
con = [
    1, 2, 3, 4
]

# train
train3['class'] = np.select(cri, con, default = train3['class'])

#### 3-6) 후보정 처리
- 강수 여부, class와 같은 변수의 경우 정수로 입력되는 값을 받아야 할것이다. 따라서 후보정 처리가 들어가야 하는 변수를 조절해준다.
- 방향과 같은 경우 0 ~ 360 사이로 입력되게 만들어주어야 한다.

In [130]:
# 강수여부 처리
train3['re'] = np.where(train3['re'] >= 0.5, 1, 0)
test2['re'] = np.where(test2['re'] >= 0.5, 1, 0)

In [138]:
# 클래스 데이터 형태 변환
train3['class'] = train3['class'].astype(int)
# test2['class'] = test2['class'].astype(int)

### 4. 파생변수 생성하기

#### 4-1) 이슬점 온도

- 안개 생기는 기준점을 미리 만들어두자
- 이슬점은 지면온도와 기온을 고려했을 때 안개가 생성되게 하는 가장 좋은 기준점이다.

In [139]:
# hm이 완전 0인경우 0.00001 더하자
train3['hm'][train3['hm'] == 0.0] = 0.00001
test2['hm'][test2['hm'] == 0.0] = 0.00001

In [140]:
# Magnus 공식 상수
a = 17.27
b = 237.7

# 알파 값 계산
train3['alpha'] = (a * train3['ta']) / (b + train3['ta']) + np.log(train3['hm'] / 100.0)
test2['alpha'] = (a * test2['ta']) / (b + test2['ta']) + np.log(test2['hm'] / 100.0)

# 이슬점온도 계산
train3['dew_point'] = (b * train3['alpha']) / (a - train3['alpha'])
test2['dew_point'] = (b * test2['alpha']) / (a - test2['alpha'])

#### 4-2) 시간적 사이클

- 계절에 따른 차이가 존재할 수 있기 때문에 계절에 관한 사이클을 만들어 주도록 하자
- 하루 단위로 온도의 변화가 존재하기 때문에 이를 학습시키기 위해 하루 단위를 변화하도록 하자

In [147]:
# 시간 사이클 변수
train3['sin_time'] = np.sin(2 * np.pi * train3['time'] / 24)
train3['cos_time'] = np.cos(2 * np.pi * train3['time'] / 24)

test2['sin_time'] = np.sin(2 * np.pi * test2['time'] / 24)
test2['cos_time'] = np.cos(2 * np.pi * test2['time'] / 24)

In [148]:
# 계절 사이클 변수 - 월별 주기
train3['sin_month'] = np.sin(2 * np.pi * train3['month'] / 12)
train3['cos_month'] = np.cos(2 * np.pi * train3['month'] / 12)

test2['sin_month'] = np.sin(2 * np.pi * test2['month'] / 12)
test2['cos_month'] = np.cos(2 * np.pi * test2['month'] / 12)

#### 4-3) 안개 발생 조건

- 다음 생성할 변수는 세가지 영역으로 나눠서 형성할 것이다.

> 1. 안개 생성 조건: 지역별로 생성되는 조건이 상이하기 때문에 안개별로 조건을 달리하여 적용할 것이다.
> 2. 안개 지속 조건: 얼마나 지속될지, 언제까지 지속되는 조건인지에 따라 계속해서 발생하고 있을 수 있기 때문에 적용한다. 조건이 같기 때문에 통일된 변수로 제작한다.
> 3. 안개 짙어짐 조건: 마찬가지로 조건은 거의 동일하다 보면 된다. 

- 안개 종류에 따른 지형
    - 복사: A내륙, B산간
    - 이류: C동해, D서해, E남해
    - 연안: D서해
    - 전선: C동해
    - 해무: E남해
    - 활승: B산간
    - 김: B산간

In [151]:
# 온도조건 미리 계산하기
train3['diff_air-dew'] = train3['ta'] - train3['dew_point']
train3['diff_ts-dew'] = train3['ts'] - train3['dew_point']

test2['diff_air-dew'] = test2['ta'] - test2['dew_point']
test2['diff_ts-dew'] = test2['ts'] - test2['dew_point']

복사안개

In [156]:
# 안개 생성 조건 1. 복사안개
# train
cri = [
    (train3['ground'].isin(['A', 'B'])) & # 내륙산간지방
    (train3['diff_ts-dew'] <= 0) & # 지면온도가 이슬점보다 같거나 낮은곳
    (train3['hm'] >= 85) & # 습도 85% 이상
    (train3['ws10_ms'] <= 2) & # 풍속 2m/s 이하
    (train3['re'] == 0) & # 강수 없음
    (train3['sun10'] == 0) # 일사 없음
]

con = [
    1
]

train3['make_copyfog'] = np.select(cri, con, default = 0)

In [157]:
# 안개 생성 조건 1. 복사안개
# test
cri = [
    (test2['ground'].isin(['A', 'B'])) & # 내륙산간지방
    (test2['diff_ts-dew'] <= 0) & # 지면온도가 이슬점보다 같거나 낮은곳
    (test2['hm'] >= 85) & # 습도 85% 이상
    (test2['ws10_ms'] <= 2) & # 풍속 2m/s 이하
    (test2['re'] == 0) & # 강수 없음
    (test2['sun10'] == 0) # 일사 없음
]

con = [
    1
]

test2['make_copyfog'] = np.select(cri, con, default = 0)

이류안개

In [158]:
# 안개 생성 조건 2. 이류안개
# train
cri = [
    (train3['ground'].isin(['C', 'D', 'E'])) & # 해안가 지역
    (train3['diff_ts-dew'] <= 0) & # 지표면이 이슬점과 같거나 아래
    (train3['hm'] >= 85) & # 습도 85%이상
    (train3['ws10_ms'] >= 2) & # 풍속 2m/s이상
    (train3['ws10_ms'] <= 15) # 풍속 15m/s 이하
]

con = [
    1
]

train3['make_irufog'] = np.select(cri, con, default = 0)

In [159]:
# 안개 생성 조건 2. 이류안개
# test
cri = [
    (test2['ground'].isin(['C', 'D', 'E'])) & # 해안가 지역
    (test2['diff_ts-dew'] <= 0) & # 지표면이 이슬점과 같거나 아래
    (test2['hm'] >= 85) & # 습도 85%이상
    (test2['ws10_ms'] >= 2) & # 풍속 2m/s이상
    (test2['ws10_ms'] <= 15) # 풍속 15m/s 이하
]

con = [
    1
]

test2['make_irufog'] = np.select(cri, con, default = 0)

연안안개

In [160]:
# 안개 생성 조건 3. 연안안개
# train
cri = [
    (train3['ground'] == 'D') & # 서해안
    (train3['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (train3['hm'] >= 85) & # 습도 85%이상
    (train3['ws10_ms'] >= 2) & # 풍속 2 ~ 10
    (train3['ws10_ms'] <= 10) & 
    (train3['ws10_deg'] >= 45) & # 방향 서 -> 동
    (train3['ws10_deg'] <= 135) & 
    (train3['re'] == 0) # 날씨가 흐리지 않게 조절
]

con = [
    1
]

train3['make_westfog'] = np.select(cri, con, default = 0)

In [161]:
# 안개 생성 조건 3. 연안안개
# test
cri = [
    (test2['ground'] == 'D') & 
    (test2['diff_ts-dew'] <= 0) & 
    (test2['hm'] >= 85) & 
    (test2['ws10_ms'] >= 2) & 
    (test2['ws10_ms'] <= 10) & 
    (test2['ws10_deg'] >= 45) & 
    (test2['ws10_deg'] <= 135) & 
    (test2['re'] == 0)
]

con = [
    1
]

test2['make_westfog'] = np.select(cri, con, default = 0)

전선안개

In [162]:
# 안개 생성 조건 4. 전선안개
# train
cri = [
    (train3['ground'] == 'C') & # 동해안
    (train3['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (train3['hm'] >= 85) & # 습도 85%이상
    (train3['ws10_ms'] >= 2) & # 풍속 2 ~ 15
    (train3['ws10_ms'] <= 15) & 
    (train3['ws10_deg'] >= 225) & # 방향 동 -> 서
    (train3['ws10_deg'] <= 315) & 
    (train3['re'] == 1) # 날씨가 구름이 많음
]

con = [
    1
]

train3['make_eastfog'] = np.select(cri, con, default = 0)

In [163]:
# 안개 생성 조건 4. 전선안개
# test
cri = [
    (test2['ground'] == 'C') & # 동해안
    (test2['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (test2['hm'] >= 85) & # 습도 85%이상
    (test2['ws10_ms'] >= 2) & # 풍속 2 ~ 15
    (test2['ws10_ms'] <= 15) & 
    (test2['ws10_deg'] >= 225) & # 방향 동 -> 서
    (test2['ws10_deg'] <= 315) & 
    (test2['re'] == 1) # 날씨가 구름이 많음
]

con = [
    1
]

test2['make_eastfog'] = np.select(cri, con, default = 0)

해무안개

In [164]:
# 안개 생성 조건 5. 해무안개
# train
cri = [
    (train3['ground'] == 'E') & # 남해안
    (train3['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (train3['hm'] >= 85) & # 습도 85%이상
    (train3['ws10_ms'] >= 2) & # 풍속 2 ~ 15
    (train3['ws10_ms'] <= 15) & 
    ((train3['ws10_deg'] >= 315) | # 방향 동 -> 서
    (train3['ws10_deg'] <= 45)) & 
    (train3['re'] == 0) # 날씨가 흐리지 않게 조절
]

con = [
    1
]

train3['make_southfog'] = np.select(cri, con, default = 0)

In [165]:
# 안개 생성 조건 5. 해무안개
# test
cri = [
    (test2['ground'] == 'E') & # 남해안
    (test2['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (test2['hm'] >= 85) & # 습도 85%이상
    (test2['ws10_ms'] >= 2) & # 풍속 2 ~ 15
    (test2['ws10_ms'] <= 15) & 
    ((test2['ws10_deg'] >= 315) | # 방향 동 -> 서
    (test2['ws10_deg'] <= 45)) & 
    (test2['re'] == 0) # 날씨가 흐리지 않게 조절
]

con = [
    1
]

test2['make_southfog'] = np.select(cri, con, default = 0)

활승안개

In [166]:
# 안개 생성 조건 6. 활승안개
# train
cri = [
    (train3['ground'] == 'B') & # 산간지방
    (train3['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (train3['hm'] >= 85) & # 습도 85%이상
    (train3['ws10_ms'] >= 2) & # 풍속 2 ~ 10
    (train3['ws10_ms'] <= 10)
]

con = [
    1
]

train3['make_mountfog'] = np.select(cri, con, default = 0)

In [167]:
# 안개 생성 조건 1. 복사안개
# test
cri = [
    (test2['ground'] == 'B') & # 산간지방
    (test2['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (test2['hm'] >= 85) & # 습도 85%이상
    (test2['ws10_ms'] >= 2) & # 풍속 2 ~ 10
    (test2['ws10_ms'] <= 10)
]

con = [
    1
]

test2['make_mountfog'] = np.select(cri, con, default = 0)

김안개

In [168]:
# 안개 생성 조건 7. 김안개
# train
cri = [
    (train3['ground'] == 'B') & # 산간지방
    (train3['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (train3['hm'] >= 85) & # 습도 85%이상
    (train3['ws10_ms'] <= 2) & # 풍속 2 이하
    (train3['re'] == 0) # 날씨가 흐리지 않게 조절
]

con = [
    1
]

train3['make_gimfog'] = np.select(cri, con, default = 0)

In [169]:
# 안개 생성 조건 7. 김안개
# test
cri = [
    (test2['ground'] == 'B') & # 산간지방
    (test2['diff_ts-dew'] <= 0) & # 지면이 이슬점보다 같거나 낮게
    (test2['hm'] >= 85) & # 습도 85%이상
    (test2['ws10_ms'] <= 2) & # 풍속 2 이하
    (test2['re'] == 0) # 날씨가 흐리지 않게 조절
]

con = [
    1
]

test2['make_gimfog'] = np.select(cri, con, default = 0)

지속조건

In [170]:
# 안개 지속 조건 
cri = [
    (train3['diff_air-dew'] <= 2) & # 기온이 이슬점보다 같거나 낮게
    (train3['hm'] >= 90) & # 습도 90%이상
    (train3['ws10_ms'] <= 2) & # 풍속 2 이하
    (train3['re'] == 0) & # 날씨가 흐리지 않게 조절
    (train3['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

train3['retain_fog'] = np.select(cri, con, default = 0)

In [171]:
# 안개 지속 조건 
# test
cri = [
    (test2['diff_air-dew'] <= 2) & # 기온이 이슬점보다 같거나 낮게
    (test2['hm'] >= 90) & # 습도 90%이상
    (test2['ws10_ms'] <= 2) & # 풍속 2 이하
    (test2['re'] == 0) & # 날씨가 흐리지 않게 조절
    (test2['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

test2['retain_fog'] = np.select(cri, con, default = 0)

가속조건

In [172]:
# 안개 가속 조건 
cri = [
    (train3['diff_air-dew'] <= 1) & # 기온이 이슬점보다 같거나 낮게
    (train3['hm'] >= 95) & # 습도 95%이상
    (train3['ws10_ms'] <= 1) & # 풍속 1 이하
    (train3['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

train3['upclass_fog'] = np.select(cri, con, default = 0)

In [173]:
# 안개 가속 조건 
# test
cri = [
    (test2['diff_air-dew'] <= 1) & # 기온이 이슬점보다 같거나 낮게
    (test2['hm'] >= 95) & # 습도 95%이상
    (test2['ws10_ms'] <= 1) & # 풍속 1 이하
    (test2['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

test2['upclass_fog'] = np.select(cri, con, default = 0)

In [175]:
# save the data
train3.to_csv('../data/train_make_columns.csv', index = False)
test2.to_csv('../data/test_make_columns.csv', index = False)

In [176]:
train3.columns

Index(['class', 'day', 'hm', 'minute', 'month', 're', 'stn_id', 'sun10', 'ta',
       'time', 'ts', 'vis1', 'ws10_deg', 'ws10_ms', 'year', 'ground', 'hour',
       'DateTime', 'alpha', 'dew_point', 'sin_time', 'cos_time', 'sin_month',
       'cos_month', 'diff_air-dew', 'diff_ts-dew', 'make_copyfog',
       'make_irufog', 'make_westfog', 'make_eastfog', 'make_southfog',
       'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog'],
      dtype='object')

In [180]:
for c in ['make_copyfog', 'make_irufog', 'make_westfog', 'make_eastfog', 'make_southfog',
       'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog', 'class']:
    
    display(pd.crosstab(train3[c], train3['class']))

class,1,2,3,4
make_copyfog,,,,
0,7584,11810,11661,3097184
1,264,261,528,27168


class,1,2,3,4
make_irufog,,,,
0,7754,11965,11899,3115501
1,94,106,290,8851


class,1,2,3,4
make_westfog,,,,
0,7848,12066,12173,3124043
1,0,5,16,309


class,1,2,3,4
make_eastfog,,,,
0,7848,12070,12183,3124303
1,0,1,6,49


class,1,2,3,4
make_southfog,,,,
0,7848,12071,12188,3124216
1,0,0,1,136


class,1,2,3,4
make_mountfog,,,,
0,7826,12038,12090,3121353
1,22,33,99,2999


class,1,2,3,4
make_gimfog,,,,
0,7791,11956,11973,3115651
1,57,115,216,8701


class,1,2,3,4
retain_fog,,,,
0,2430,3961,6884,2820463
1,5418,8110,5305,303889


class,1,2,3,4
upclass_fog,,,,
0,4436,6758,8511,3002771
1,3412,5313,3678,121581


class,1,2,3,4
class,,,,
1,7848,0,0,0
2,0,12071,0,0
3,0,0,12189,0
4,0,0,0,3124352


### 오늘의 결론
- 할루시네이션 제대로 당했다 .... 다시 찾자